# Ausgleichsenergiepreisprognose
Dieses Notebook sammelt Daten und entwickelt ein erstes Basismodell zur Vorhersage der Schweizer Ausgleichsenergiepreise.

## Datenbeschaffung
Die historischen positiven und negativen Ausgleichsenergiepreise werden monatlich von Swissgrid als Excel-Dateien bereitgestellt. Im Folgenden wird gezeigt, wie diese automatisch heruntergeladen und eingelesen werden können. Bei fehlender Internetverbindung wird ein Platzhalter erzeugt.

In [ ]:

import pandas as pd
from pathlib import Path
import requests

BASE_URL = "https://www.swissgrid.ch/dam/swissgrid/current/Data/excel/balancingenergy/"

def download_monthly_price(year: int, month: int, folder: Path = Path('data')) -> Path:
    folder.mkdir(exist_ok=True)
    filename = folder / f"{year}_{month:02d}_balancing_energy_prices.xlsx"
    url = f"{BASE_URL}{year}/{month:02d}/balancing_energy_prices_{year}_{month:02d}.xlsx"
    try:
        r = requests.get(url)
        r.raise_for_status()
        filename.write_bytes(r.content)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Could not download {url}: {e}
Generating fake data instead.")
        dummy = pd.DataFrame({
            'Time': pd.date_range(f"{year}-{month:02d}-01", periods=96, freq='15min'),
            'PositivePrice': 0.0,
            'NegativePrice': 0.0
        })
        dummy.to_excel(filename, index=False)
    return filename


Nach dem Herunterladen werden die Daten in einen DataFrame geladen:

In [ ]:

def load_price_file(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    df = df.rename(columns=str.strip)
    df['Time'] = pd.to_datetime(df['Time'])
    return df.set_index('Time')

# Beispiel: Preisdatei fuer Januar 2024 herunterladen und einlesen
file_path = download_monthly_price(2024, 1)
prices = load_price_file(file_path)
prices.head()


## Feature Engineering
Im Leitfaden sind zahlreiche potentielle Prädiktoren genannt. Für den Start erzeugen wir einfache Zeitfeatures und Lag-Variablen.

In [ ]:

def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hour'] = df.index.hour
    df['weekday'] = df.index.weekday
    df['month'] = df.index.month
    df['dayofyear'] = df.index.dayofyear
    df['lag_price'] = df['PositivePrice'].shift(1)
    df['rolling_mean_24h'] = df['PositivePrice'].rolling(window=24).mean()
    df['rolling_std_24h'] = df['PositivePrice'].rolling(window=24).std()
    return df

features = add_basic_features(prices)
features.dropna().head()


## Modellansatz
Für ein Minimalbeispiel verwenden wir ein Gradient Boosting Regressor auf Stundenbasis. Das Ziel ist der PositivePrice.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

X = features[['hour', 'weekday', 'month', 'dayofyear', 'lag_price', 'rolling_mean_24h', 'rolling_std_24h']].dropna()
y = features.loc[X.index, 'PositivePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))


Das Notebook stellt damit eine erste Vorlage dar, die nach Verfügbarkeit echter Daten weiter ausgebaut werden kann.